In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_validate
import re
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
import re
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [2]:
data = pd.read_csv('processed_df.csv')

**ВЫКИНЕМ НАНЫ**

In [3]:
data_no_nans = data.copy()
data_no_nans = data_no_nans.dropna()
data_no_nans.isna().sum()

Maruti                  0
Skoda                   0
Honda                   0
Hyundai                 0
Toyota                  0
Ford                    0
Renault                 0
Mahindra                0
Tata                    0
Chevrolet               0
Fiat                    0
Datsun                  0
Jeep                    0
Mercedes-Benz           0
Mitsubishi              0
Audi                    0
Volkswagen              0
BMW                     0
Nissan                  0
Lexus                   0
Jaguar                  0
Land                    0
MG                      0
Volvo                   0
Daewoo                  0
Kia                     0
Force                   0
Ambassador              0
Ashok                   0
Isuzu                   0
Opel                    0
year                    0
selling_price           0
km_driven               0
Diesel                  0
Individual              0
Dealer                  0
Manual                  0
First Owner 

**ВЫДЕЛИМ ТРЕНИРОВОЧНЫЕ И ТЕСТОВЫЕ МАССИВЫ**

In [4]:
train_data_no_nans = data_no_nans.sample(frac=0.8)
test_data_no_nans = data_no_nans.iloc[~data_no_nans.index.isin(train_data_no_nans.index), :]

In [5]:
features_train = train_data_no_nans.iloc[:, train_data_no_nans.columns != 'selling_price']
responses_train = train_data_no_nans['selling_price'].to_frame(name='selling_price')
features_test = test_data_no_nans.iloc[:, test_data_no_nans.columns != 'selling_price']
responses_test = test_data_no_nans['selling_price'].to_frame(name='selling_price')

**НОРМИРУЕМ ДАННЫЕ**

In [6]:
scaller = StandardScaler()
x_train = scaller.fit_transform(features_train)
y_train = scaller.fit_transform(responses_train)
x_test = scaller.fit_transform(features_test)
y_test = scaller.fit_transform(responses_test)

# **ОБУЧИМ МОДЕЛЬ НА ТЕХ ДАННЫХ, ЧТО ЕСТЬ**

**ОБУЧИМ МОДЕЛЬ С ЛИНЕАРИЗАЦИЕЙ КОЭФФИЦИЕНТОВ L1 LASSO**

In [7]:
l1_regr = Lasso(1e-5, max_iter=10 ** 4)
l1_regr.fit(x_train, y_train)
l1_regr.score(x_train, y_train)

0.852654394289907

In [8]:
l1_regr.coef_

array([ 4.63707436e-02, -1.57320908e-02,  1.22322735e-03, -0.00000000e+00,
        8.12341922e-02, -2.68545681e-03, -2.49101713e-03, -1.36415255e-03,
       -4.47198796e-02, -1.31014913e-02, -7.86811336e-03, -1.09256245e-02,
        5.86553970e-02,  1.20975310e-01,  1.24765268e-02,  9.70966487e-02,
       -2.35318701e-02,  4.09334700e-01, -4.81318622e-03,  2.68217738e-01,
        1.74177439e-01,  9.33393069e-02,  1.16194759e-02,  1.04926231e-01,
        1.33578929e-02,  1.23310428e-02, -5.13958926e-04,  7.29391139e-03,
        7.39080684e-04,  2.00913762e-02,  5.87424218e-03,  2.15014689e-01,
       -4.18767186e-02, -1.22715430e-02,  4.67156399e-03,  3.45971229e-02,
       -4.21407011e-02, -1.36709321e+00, -1.28841701e+00, -7.17576549e-01,
       -4.14573662e-01, -1.23421053e-02,  4.89025260e-02,  2.71018605e-01,
        6.32646753e-03, -7.13050170e-02, -1.39448651e-02])

In [9]:
l1_regr.score(x_test, y_test)

0.8239398143121701

**КАК ВИДНО НА ТЕСТОВОЙ ВЫБОРКЕ РЕЗУЛЬТАТ ДАЖЕ ЛУЧШЕ.**

**МФ ПОЛУЧИЛИ РЕЗУЛЬТАТ ПО МЕТРИКЕ R2**

**ПОПРОБУЕМ ОЦЕНИТЬ С ПОМОЩЬЮ МЕТРИКИ MAE**

In [10]:
y_train_predict = l1_regr.predict(x_train)
y_test_predict = l1_regr.predict(x_test)
loss_train = mean_absolute_error(y_train, y_train_predict)
loss_test = mean_absolute_error(y_test, y_test_predict)

In [11]:
print(loss_train)
print(loss_test)

0.1965184584895035
0.21483177290895414


**ПОСМОТРИМ СКОЛЬКО ЭТО В ДЕНЕЖНОМ ЭКВИВАЛЕНТЕ С ПОМОЩЬЮ ИНВЕРСИИ НОРМАЛИЗАЦИИ**

In [12]:
print(scaller.inverse_transform([[loss_train]]))
print(scaller.inverse_transform([[loss_test]]))

[[773569.81759854]]
[[787111.25179942]]


**ОЦЕНИМ С ПОМОЩЬЮ МЕТРИКИ MAPE**

In [13]:
loss_train_mape = mean_absolute_percentage_error(y_train, y_train_predict)
loss_test_mape = mean_absolute_percentage_error(y_test, y_test_predict)

In [14]:
print(loss_train_mape)
print(loss_test_mape)

1.5102661274182543
1.4203448990423553


**ОПЯТЬ ЗАМЕЧАЕМ, ЧТО НА ТЕСТОВЫХ ДАННЫХ РЕЗУЛЬТАТ ЛУЧШЕ**

# **ДОБАВИМ ПОЛИНОМ 9 СТЕПЕНИ ПО ПРИЗНАКАМ ТИП ТОПЛИВА И ОБЬЕМ ДВИГАТЕЛЯ**

**В РЕЗУЛЬТАТЕ АНАЛИЗА ДАННЫХ (ФАЙЛ processing_nes.ipynb) МЫ ВЫЯСНИЛИ, ЧТО ОБЬЕМ ДВИГАТЕЛЯ НАХОДИТСЯ В НЕКОТОРОЙ КОРЕЛЯЦИИ ОТ ТИПА ТОПЛИВА. ПОСТРОИВ ПОЛИНОМИАЛЬНУЮ МОДЕЛЬ, ОБУЧИВ ЕЕ И ПОСЧИТАВ ПОТЕРИ С КРОСС ВАЛИДАЦИЕЙ МЫ ПОНЯЛИ, ЧТО ДЛЯ ДВУХ ЭТИХ ДВУХ ПРИЗНАКОВ С ПОЛИНОМОМ СТЕПЕНЬЮ 9 (ФАЙЛ analyse.ipynb) МЫ МОЖЕМ УЛУЧШИТЬ МОДЕЛЬ. ПОПРОБУЕМ ЕЕ УЛУЧШИТЬ ДОБАВИВ ЭТИ ПРИЗНАКИ**

**ВОЗЬМЕМ ФРЕЙМ БЕЗ НАНОВ И СКАЛЯРИЗУЕМ**

In [15]:
scaller_poly = StandardScaler()
x_poly = scaller_poly.fit_transform(data_no_nans[data_no_nans.columns.drop('selling_price')])
y_poly = scaller_poly.fit_transform(data_no_nans[['selling_price']])

In [16]:
scaled_df = pd.DataFrame(data=np.hstack((y_poly, x_poly)), columns=['selling_price'] + list(data_no_nans.columns.drop('selling_price')))
scaled_df

,selling_price,Maruti,Skoda,Honda,Hyundai,Toyota,Ford,Renault,Mahindra,Tata,...,First Owner,Second Owner,Third Owner,Fourth & Above Owner,milage,engine,max_power,torque,RPM_min,seats
0,-0.237694,1.535349,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,0.721895,-0.587260,-0.263667,-0.144741,1.021040,-0.418993,-0.495635,0.081953,-0.617447,-0.436655
1,-0.339271,-0.651318,8.627996,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,-1.385243,1.702822,-0.263667,-0.144741,0.440338,0.077846,0.345081,0.601426,-1.065356,-0.436655
2,-0.608449,-0.651318,-0.115902,3.979562,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,-1.385243,-0.587260,3.792666,-0.144741,-0.443564,0.075858,-0.381717,-0.485485,0.009625,-0.436655
3,-0.523379,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,0.721895,-0.587260,-0.263667,-0.144741,0.918261,-0.124865,-0.039962,0.337534,-0.841402,-0.436655
4,-0.644001,1.535349,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,0.721895,-0.587260,-0.263667,-0.144741,-0.854681,-0.319626,-0.091225,-0.587301,1.622096,-0.436655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7841,-0.402756,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,0.721895,-0.587260,-0.263667,-0.144741,-0.238005,-0.520348,-0.243591,-0.578644,1.174187,-0.436655
7842,-0.637652,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,-1.385243,-0.587260,-0.263667,6.908901,-0.674817,0.067909,0.529629,0.473289,-0.707029,-0.436655
7843,-0.324034,1.535349,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,...,0.721895,-0.587260,-0.263667,-0.144741,-0.032447,-0.418993,-0.498483,0.081953,-0.617447,-0.436655
7844,-0.440848,-0.651318,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,3.145985,...,0.721895,-0.587260,-0.263667,-0.144741,1.064721,-0.124865,-0.609553,-0.350941,-0.796611,-0.436655


**СОЗДАДИМ ПОЛИНОМ 9Й СТЕПЕНИ ДЛЯ ДВИГАТЕЛЯ И ДИЗЕЛЯ**

In [17]:
poly_obj = PolynomialFeatures(9)
engine_diesel_poly = poly_obj.fit_transform(scaled_df[['Diesel', 'engine']])

**УБЕРЕМ ПЕРВЫЕ ТРИ СТОЛБЦА, ПЕРВЫЙ ПОТОМУ ЧТО ЕДИНИЧНЫЙ ДЛЯ СВОБОНОГО ВЕСА, ВТОРЫЕ ДВА ПОТОМУ ЧТО ПОВТОРЕНИЕ ТЕХ, ЧТО УЖЕ ЕСТЬ В ДАТАСЕТЕ, ЗАПИШЕМ ВО ФРЕЙМ**

In [18]:
engine_diesel_poly_df = pd.DataFrame(data=engine_diesel_poly[:, 3:], columns=[f'Dies*Engine{i + 1}' for i in range(engine_diesel_poly.shape[1] - 3)])

**СОБЕРЕМ ПОЛИНОМ С ДАТАСЕТОМ**

In [19]:
poly_prepared = pd.concat([scaled_df, engine_diesel_poly_df], axis=1)

**РАЗДЕЛИМ ВЕСЬ СКАЛЯРИЗОВАННЫЙ ДАТАСЕТ НА ТРЕНИРОВОЧНЫЙ И ТЕСТОВЫЙ**

In [20]:
poly_prepared_train = poly_prepared.sample(frac=0.8, random_state=0).sort_index()
poly_prepared_test = poly_prepared.iloc[~scaled_df.index.isin(poly_prepared_train.index)].sort_index()

**НАЧНЕМ ОБУЧАТЬ МОДЕЛЬ С L1/L2 НА ТРЕНИРОВОЧНОМ ДАТАСЕТЕ**

In [ ]:
regr_poly = Lasso(alpha=0.0001, max_iter=10 ** 5, tol=0.001)
regr_poly.fit(poly_prepared_train.iloc[:, 1:], poly_prepared_train.iloc[:, 0])

In [22]:
regr_poly.coef_

array([ 2.85356497e-02, -2.93534865e-03,  4.97558079e-04, -1.09350179e-02,
        7.16913327e-02, -9.02386103e-03, -1.29819911e-02,  0.00000000e+00,
       -4.44972480e-02, -1.81780996e-02, -1.04448208e-02, -1.46821334e-02,
        2.45006506e-02,  1.24413497e-01,  1.10478875e-02,  8.31205106e-02,
       -2.43948309e-02,  3.89839682e-01, -5.71938664e-03,  2.63924398e-01,
        1.69053451e-01,  7.69334254e-02,  1.03837045e-02,  1.04773883e-01,
        8.80731128e-03,  1.57260379e-02, -1.23890958e-03,  1.54161791e-02,
        0.00000000e+00,  2.29437793e-02,  6.70287119e-03,  2.17449085e-01,
       -6.52608431e-02,  0.00000000e+00, -2.41473140e-03,  3.44662701e-02,
       -2.67660637e-02, -1.40981635e+00, -1.33240131e+00, -7.40168563e-01,
       -4.23074495e-01, -3.41647764e-02, -0.00000000e+00,  3.28802955e-01,
        1.22725489e-02, -5.72594117e-02, -4.79398361e-03, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00, -

In [23]:
poly_prepared_train.iloc[:, 1:]

,Maruti,Skoda,Honda,Hyundai,Toyota,Ford,Renault,Mahindra,Tata,Chevrolet,...,Dies*Engine43,Dies*Engine44,Dies*Engine45,Dies*Engine46,Dies*Engine47,Dies*Engine48,Dies*Engine49,Dies*Engine50,Dies*Engine51,Dies*Engine52
1,-0.651318,8.627996,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,-0.173,...,0.408456,0.035122,0.003020,0.000260,0.000022,1.920186e-06,1.651136e-07,1.419785e-08,1.220850e-09,1.049789e-10
2,-0.651318,-0.115902,3.979562,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,-0.173,...,-2.448245,0.168132,-0.011546,0.000793,-0.000054,3.739690e-06,-2.568218e-07,1.763713e-08,-1.211223e-09,8.318025e-11
3,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,-0.173,...,0.408456,-0.056337,0.007770,-0.001072,0.000148,-2.038798e-05,2.812031e-06,-3.878519e-07,5.349483e-08,-7.378322e-09
4,1.535349,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,-0.173,...,-2.448245,-0.708419,-0.204987,-0.059315,-0.017163,-4.966285e-03,-1.437034e-03,-4.158171e-04,-1.203200e-04,-3.481553e-05
5,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,-0.173,...,-2.448245,-1.153302,-0.543289,-0.255929,-0.120561,-5.679309e-02,-2.675368e-02,-1.260293e-02,-5.936901e-03,-2.796713e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7840,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,-0.173,...,0.408456,-0.056337,0.007770,-0.001072,0.000148,-2.038798e-05,2.812031e-06,-3.878519e-07,5.349483e-08,-7.378322e-09
7841,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,-0.173,...,-2.448245,-1.153302,-0.543289,-0.255929,-0.120561,-5.679309e-02,-2.675368e-02,-1.260293e-02,-5.936901e-03,-2.796713e-03
7842,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,-0.317865,-0.173,...,0.408456,0.030639,0.002298,0.000172,0.000013,9.700712e-07,7.276715e-08,5.458421e-09,4.094480e-10,3.071358e-11
7844,-0.651318,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,3.145985,-0.173,...,0.408456,-0.056337,0.007770,-0.001072,0.000148,-2.038798e-05,2.812031e-06,-3.878519e-07,5.349483e-08,-7.378322e-09
